In [1]:
import lightgbm as lgb
import pandas as pd
import numpy as np
from hyperopt import fmin, tpe, hp
import ipynb.fs.full.utils as utils
import ipynb.fs.full.features as features
import ipynb.fs.full.features_distancias as f_distancias

df_train = pd.read_csv('./data/train_filtrado.csv')
# df_train = utils.dolarizar_df(df_train)

# Para usarse con el submit a Kaggle
df_test = pd.read_csv('./data/test.csv')

df_train = features.llenar_nulls(df_train)
df_test = features.llenar_nulls(df_test, hgb_mean=True, df_fill=df_train)

# df_train_cluster = pd.read_csv('./data/clustering_train.csv').rename(columns={'label': 'clustering_label'})
# df_test_cluster = pd.read_csv('./data/clustering_test.csv').rename(columns={'label': 'clustering_label'})

# df_train = pd.merge(df_train, df_train_cluster, on='id')
# df_test = pd.merge(df_test, df_test_cluster, on='id')

df_train_idf = pd.read_csv('./data/train_idf.csv')
df_test_idf = pd.read_csv('./data/test_idf.csv')

df_train = pd.merge(df_train, df_train_idf, on= 'id', how= 'left')
df_test = pd.merge(df_test, df_test_idf, on= 'id', how= 'left')

# df_train_neigh = pd.read_csv('./data/precios_distancias_2k_train.csv')
# # df_test_neigh = pd.read_csv('./data/precios_distancias_2k_test.csv')
# df_train = pd.merge(df_train, df_train_neigh, on= 'id', how= 'left')
# df_test = pd.merge(df_test, df_test_neigh, on= 'id', how= 'left')

# df_train, df_test = utils.dividir_df_testeo(df_train, test_size=0.3)

In [7]:
titulo_train = pd.read_csv('./data/train_titulos_caros.csv')
titulo_test = pd.read_csv('./data/test_titulos_caros.csv')

train_DK = pd.read_csv('./data/df_train_KD.csv')
test_DK = pd.read_csv('./data/df_test_KD.csv')

titulo_train = titulo_train.drop(['titulo', 'descripcion', 'tipodepropiedad', 'direccion', 'ciudad',
       'provincia', 'antiguedad', 'habitaciones', 'garages', 'banos',
       'metroscubiertos', 'metrostotales', 'idzona', 'lat', 'lng', 'fecha',
       'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas',
       'centroscomercialescercanos', 'precio'], axis = 1, inplace = False)

titulo_test = titulo_test.drop(['titulo', 'descripcion', 'tipodepropiedad', 'direccion', 'ciudad',
       'provincia', 'antiguedad', 'habitaciones', 'garages', 'banos',
       'metroscubiertos', 'metrostotales', 'idzona', 'lat', 'lng', 'fecha',
       'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas',
       'centroscomercialescercanos'], axis = 1, inplace = False)

test_DK = test_DK.drop(['titulo', 'descripcion', 'tipodepropiedad', 'direccion', 'ciudad',
       'provincia', 'antiguedad', 'habitaciones', 'garages', 'banos',
       'metroscubiertos', 'metrostotales', 'idzona', 'lat', 'lng', 'fecha',
       'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas',
       'centroscomercialescercanos'], axis = 1, inplace = False)

In [8]:
df_train = pd.merge(df_train, titulo_train, on= 'id', how= 'left')
df_test = pd.merge(df_test, titulo_test, on= 'id', how= 'left')

df_train = pd.merge(df_train, train_DK, on= 'id', how= 'left')
df_test = pd.merge(df_test, test_DK, on= 'id', how= 'left')

In [9]:
df_test_f = features.features_independientes_precio(df_test)
df_test_f = features.features_dependientes_precio(df_test_f, df_train)

df_train_f = features.features_independientes_precio(df_train)
df_train_f = features.features_dependientes_precio(df_train_f, df_train)

df_test_f, cols_tipodepropiedad_ohe = features.columna_a_ohe(df_test_f, 'tipodepropiedad', N=100, df_aux=df_train, devolver_cols=True)
df_test_f, cols_provincia_ohe = features.columna_a_ohe(df_test_f, 'provincia', N=100, df_aux=df_train, devolver_cols=True)
df_test_f, cols_zona_ohe = features.columna_a_ohe(df_test_f, 'zona', df_aux=df_train_f, devolver_cols=True)

df_train_f = features.columna_a_ohe(df_train_f, 'tipodepropiedad', N=100, df_aux=df_test)
df_train_f = features.columna_a_ohe(df_train_f, 'provincia', N=100, df_aux=df_test)
df_train_f = features.columna_a_ohe(df_train_f, 'zona', df_aux=df_test_f)


df_train_f['fecha'] = pd.to_datetime(df_train_f['fecha']).astype(int)
df_test_f['fecha'] = pd.to_datetime(df_test_f['fecha']).astype(int)


df_train_f = f_distancias.feature_distancias(df_train_f)
df_test_f = f_distancias.feature_distancias(df_test_f, df_train_f)

df_train_f = features.rankings(df_train_f, df_train_f)
df_test_f = features.rankings(df_test_f, df_train_f)


In [ ]:
# df_train_f, df_test_f = utils.dividir_df_testeo(df_train_f, test_size=0.3)

## Selector de Features

In [11]:
from sklearn.base import BaseEstimator

class FeatureSelector(BaseEstimator):
    
    base_features = list(df_train_f.drop('precio', axis=1, errors='ignore').columns)
    
    def __init__(self, features):
        base_features = FeatureSelector.base_features

        self.features = features
        self.features_index = [i for i in range(len(base_features)) if base_features[i] in features]
        
    def fit(self, x, y=None):
        return self

    def transform(self, x):
        return x[:, self.features_index]

## LightGBM model

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

class LightGBMWrapper(lgb.LGBMRegressor):
    
    def fit(self, x, y):
        x_train, x_eval, y_train, y_eval = train_test_split(x, y, test_size=0.01, random_state=0)
        eval_set = [(x_eval, y_eval)]
        if self.boosting_type == 'dart':
            x_train = x
            y_train = y
            eval_set = None

        return super(LightGBMWrapper, self).fit(x_train, y_train, 
                                                early_stopping_rounds=15, eval_set=eval_set)
    
    def predict(self, X):
        return super(LightGBMWrapper, self).predict(X)

params = {
 'bagging_fraction': 0.5079900308614034,
 'bagging_freq': 0,
 'max_bin': 1462,
 'max_depth': 22,
 'min_data_in_leaf': 58,
 'min_split_gain': 0.6819500375795696,
 'num_leaves': 668,
 'reg_lambda': 48.786835826989865,
 'boosting_type': 'gbdt',
 'metric': 'mae',
 'n_jobs': 4,
 'objective': 'regression',
 'n_estimators': 1500,
}

lgb_m = Pipeline(steps=[
    ('feature_selector', FeatureSelector(['habitaciones', 'garages', 'banos','antiguedad',
           'metroscubiertos', 'metrostotales','lat_norm', 'lng_norm','gimnasio', 'usosmultiples', 
            'piscina','prop_frecuente', 'top_provincia', 'promedio_precio_ciudad', 
            'anio', 'promedio_id_zona', 'promedio_precio_tipo_propiedad', 'count_id_zona',
            'count_ciudad', 'puntaje', 'count_tipo_propiedad_ciudad', 'fecha',
            'promedio_precio_tipo_propiedad_ciudad_gen','dias_desde_datos',
            'meses_desde_datos', 'porcentaje_metros', 'distancia_ciudad_centrica',
            'tam_ambientes','idf_titulo','idf_descripcion', 'distancia_ciudad_cara', 'similares_count',
       'densidad', 'ranking_en_provincia_tipodepropiedad_precio',
       'ranking_en_provincia_tipodepropiedad_cantidad']
        )),
    ('lightgbm', LightGBMWrapper(**params))
])

## XGBoost

In [14]:
import xgboost as xgb

class XGBoostWrapper(xgb.XGBRegressor):
    
    def fit(self, x, y):
        x_train, x_eval, y_train, y_eval = train_test_split(x, y, test_size=0.05, random_state=0)
        eval_set = [(x_eval, y_eval)]

        return super(xgb.XGBRegressor, self).fit(x_train, y_train, 
                                                early_stopping_rounds=10, eval_set=eval_set)
    
    def predict(self, X):
        return super(xgb.XGBRegressor, self).predict(X)

hps = {
 'verbosity': 0,
 'objective': 'reg:squarederror',
 'eval_metric': 'mae',
 'n_estimators': 1000,
 'n_jobs': 4,
 'reg_alpha': 20.91434940058063,
 'colsample_bytree': 0.65,
 'learning_rate': 0.14,
 'max_depth': int(16.0),
#  'scale_pos_weight': 5,
#  'subsample': 0.9569019167459879
}

xgb_m = Pipeline([
    ('feature_selector', FeatureSelector(['habitaciones', 'garages', 'banos','antiguedad',
           'metroscubiertos', 'metrostotales','lat_norm', 'lng_norm','gimnasio', 'usosmultiples', 
            'piscina','prop_frecuente', 'top_provincia', 'promedio_precio_ciudad', 
            'anio', 'promedio_id_zona', 'promedio_precio_tipo_propiedad', 'count_id_zona',
            'count_ciudad', 'puntaje', 'count_tipo_propiedad_ciudad', 'fecha',
            'promedio_precio_tipo_propiedad_ciudad_gen','dias_desde_datos',
            'meses_desde_datos', 'porcentaje_metros', 'distancia_ciudad_centrica',
            'tam_ambientes','idf_titulo','idf_descripcion', 'distancia_ciudad_cara', 'similares_count',
       'densidad', 'ranking_en_provincia_tipodepropiedad_precio',
       'ranking_en_provincia_tipodepropiedad_cantidad'])),
    ('xgboost', XGBoostWrapper(**hps))
])

## RandomForest

In [15]:
from sklearn.ensemble import RandomForestRegressor

params = {'bootstrap': False,
          'max_features': 'sqrt',
          'min_samples_split': 4,
          'n_jobs': 2,
          'n_estimators': 100}

forest_m = Pipeline([
    ('feature_selector', FeatureSelector(['garages', 'banos', 'metroscubiertos', 'metrostotales',
       'lat', 'lng', 'fecha', 'porcentaje_metros', 'diferencia_metros',
       'metroscubiertos_bins_unif', 'metroscubiertos_bins_perc',
       'metros_totales_normalizados', 'metros_cubiertos_normalizados',
       'promedio_metros_tipo_propiedad',
       'promedio_metros_cubiertos_provincia', 'mes', 'dias_desde_datos', 'tam_ambientes',
       'promedio_precio_provincia', 'promedio_precio_ciudad', 'count_ciudad',
       'promedio_id_zona', 'count_id_zona', 'promedio_precio_tipo_propiedad',
       'promedio_precio_tipo_propiedad_ciudad',
       'count_tipo_propiedad_ciudad', 'promedio_por_mes', 'promedio_precio_hbg_tipo_propiedad',
       'puntaje', 'distancia_ciudad_centrica',
       'distancia_centro_mexico',
       'ranking_en_provincia_tipodepropiedad_precio',
       'ranking_en_provincia_tipodepropiedad_cantidad'])),
    ('random_forest', RandomForestRegressor(**params))
])

## ExtraTrees

In [16]:
from sklearn.ensemble import ExtraTreesRegressor

params = {'criterion': 'mse',
 'max_features': 'sqrt',
 'min_samples_split': 4}

extratrees_m =  Pipeline([
    ('feature_selector', FeatureSelector(['antiguedad', 'garages', 'banos', 'metroscubiertos', 'metrostotales',
       'lat', 'lng', 'fecha', 'idf_titulo', 'idf_descripcion',
       'peso_descripcion', 'porcentaje_metros', 'diferencia_metros',
       'metroscubiertos_bins_unif', 'metroscubiertos_bins_perc',
       'metros_totales_normalizados', 'metros_cubiertos_normalizados',
       'promedio_metros_tipo_propiedad', 'promedio_metros_cub_tipo_propiedad',
       'promedio_metros_cubiertos_provincia', 'mes', 'dia', 'dias_desde_datos',
       'meses_desde_datos', 'antiguedad_bins_perc', 'tam_ambientes',
       'promedio_precio_provincia', 'promedio_precio_ciudad',
       'promedio_precio_ciudad_gen', 'varianza_precio_ciudad', 'count_ciudad',
       'promedio_id_zona', 'promedio_id_zona_gen', 'varianza_id_zona',
       'count_id_zona', 'promedio_precio_tipo_propiedad',
       'promedio_precio_tipo_propiedad_ciudad',
       'promedio_precio_tipo_propiedad_ciudad_gen',
       'count_tipo_propiedad_ciudad', 'promedio_por_mes', 'varianza_por_mes',
       'promedio_precio_habitaciones_banos_garages',
       'promedio_precio_banos_garages', 'promedio_precio_hbg_tipo_propiedad',
       'lat_norm', 'lng_norm', 'puntaje', 'distancia_ciudad_centrica',
       'distancia_centro_mexico', 'distancia_ciudad_cara',
       'ranking_en_provincia_tipodepropiedad_precio',
       'ranking_en_provincia_tipodepropiedad_cantidad'])),
    ('extra_trees', ExtraTreesRegressor(n_estimators=50, **params))
])

## CatBoost

In [17]:
from catboost import CatBoostRegressor


class CatBoostWrapper(CatBoostRegressor):
    
    def fit(self, x, y):
        x_train, x_eval, y_train, y_eval = train_test_split(x, y, test_size=0.05)
        eval_set = [(x_eval, y_eval)]

        # posiciones de features categoricas encontradas a mano
        cat_features = [0, 1, 2]
        return super(CatBoostWrapper, self).fit(x, y, cat_features=cat_features, eval_set=eval_set, early_stopping_rounds=25)


f_s = FeatureSelector(['garages', 'banos', 'metroscubiertos', 'metrostotales',
       'lat', 'lng', 'fecha', 'porcentaje_metros', 'diferencia_metros',
       'promedio_metros_tipo_propiedad', 'promedio_metros_cubiertos_provincia', 'mes', 
       'dias_desde_datos', 'tam_ambientes',
       'promedio_precio_provincia', 'promedio_precio_ciudad',
       'count_ciudad', 'promedio_id_zona',
       'count_id_zona', 'promedio_precio_tipo_propiedad',
       'promedio_precio_tipo_propiedad_ciudad',
       'count_tipo_propiedad_ciudad', 'promedio_por_mes', 'promedio_precio_hbg_tipo_propiedad',
       'puntaje', 'distancia_ciudad_centrica',
       'distancia_centro_mexico', 'provincia', 'ciudad', 'tipodepropiedad',
       'ranking_en_provincia_tipodepropiedad_precio',
       'ranking_en_provincia_tipodepropiedad_cantidad'] + ['provincia', 'ciudad', 'tipodepropiedad'])

params = {
    'od_wait': 50,
    'od_type': 'Iter',
    'learning_rate': 0.15,
    'l2_leaf_reg': 3,
    'depth': 12,
    'colsample_bylevel': 0.5,
    'border_count': 128,
    'iterations': 800,
    'silent': True,
    'eval_metric': 'MAE'
}


catboost_m = Pipeline([
    ('feature_selector', f_s),
    ('catboost', CatBoostWrapper(**params))
])

## Keras model

In [18]:
from keras.wrappers.scikit_learn import KerasRegressor
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Activation
from sklearn.preprocessing import MinMaxScaler

def keras_modelo():    
    model = Sequential()
    model.add(Dense(units=512, activation='relu'))
    model.add(Dropout(0.13))
    model.add(Dense(units=250, activation='relu'))
    model.add(Dropout(0.13))
    model.add(Dense(units=128, activation='relu'))
    model.add(Dropout(0.13))
    model.add(Dense(units=1, activation='linear'))

    model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_squared_error'])  
    return model

keras_m = Pipeline([
    ('feature_selector', FeatureSelector(['antiguedad', 'habitaciones', 'garages', 'banos', 'metroscubiertos',
       'metrostotales', 'lat', 'lng', 'gimnasio', 'usosmultiples',
       'piscina', 'clustering_label', 'idf_descripcion', 'porcentaje_metros', 'diferencia_metros',
       'promedio_metros_tipo_propiedad', 'prop_frecuente', 'top_provincia',
       'promedio_metros_totales_provincia', 'anio', 'mes',
       'trimestre', 'dias_desde_datos', 'cantidad_inquilinos', 'tam_ambientes', 'promedio_precio_provincia',
       'promedio_precio_ciudad', 'count_ciudad', 'promedio_id_zona',
       'count_id_zona','promedio_precio_tipo_propiedad',
       'promedio_precio_tipo_propiedad_ciudad', 'count_tipo_propiedad',
       'count_tipo_propiedad_ciudad', 'promedio_por_mes', 
       'promedio_precio_banos_garages', 'promedio_precio_hbg_tipo_propiedad',
       'promedio_precio_booleanos', 'puntaje',
       'distancia_ciudad_centrica', 'distancia_centro_mexico',
       'ranking_en_provincia_tipodepropiedad_precio',
       'ranking_en_provincia_tipodepropiedad_cantidad'])),
    ('scaler', MinMaxScaler()),
    ('keras', KerasRegressor(build_fn=keras_modelo, epochs=5))
])

Using TensorFlow backend.


## Stacking

In [20]:
from sklearn.preprocessing import MinMaxScaler
from vecstack import stacking
from vecstack_sk import StackingTransformer

modelos = [
#            ('keras', keras_m),
           ('xgboost', xgb_m),
           ('lightgbm', lgb_m), 
#            ('randomforest', forest_m),
#            ('extratrees', extratrees_m),
#            ('catboost', catboost_m)
          ]

stack = StackingTransformer(modelos, 
                          regression=True, verbose=2, n_folds=5)

stack.fit(df_train_f.drop('precio', axis=1).values, 
                          df_train_f['precio'].values)

task:         [regression]
metric:       [mean_absolute_error]
variant:      [A]
n_estimators: [2]

estimator  0: [xgboost: Pipeline]
[0]	validation_0-mae:2.19764e+06
Will train until validation_0-mae hasn't improved in 10 rounds.
[1]	validation_0-mae:1.89877e+06
[2]	validation_0-mae:1.64427e+06
[3]	validation_0-mae:1.42926e+06
[4]	validation_0-mae:1.24721e+06
[5]	validation_0-mae:1.09811e+06
[6]	validation_0-mae:973400
[7]	validation_0-mae:871175
[8]	validation_0-mae:789999
[9]	validation_0-mae:723674
[10]	validation_0-mae:671219
[11]	validation_0-mae:630141
[12]	validation_0-mae:597733
[13]	validation_0-mae:572623
[14]	validation_0-mae:553789
[15]	validation_0-mae:540576
[16]	validation_0-mae:529672
[17]	validation_0-mae:521442
[18]	validation_0-mae:515304
[19]	validation_0-mae:510504
[20]	validation_0-mae:506921
[21]	validation_0-mae:504130
[22]	validation_0-mae:502229
[23]	validation_0-mae:500194
[24]	validation_0-mae:498876
[25]	validation_0-mae:497222
[26]	validation_0-mae:496599

[270]	validation_0-mae:486347
[271]	validation_0-mae:486359
[272]	validation_0-mae:486361
[273]	validation_0-mae:486359
[274]	validation_0-mae:486361
[275]	validation_0-mae:486362
[276]	validation_0-mae:486364
[277]	validation_0-mae:486364
[278]	validation_0-mae:486355
Stopping. Best iteration:
[268]	validation_0-mae:486340

    fold  0:  [479148.43055849]
[0]	validation_0-mae:2.22091e+06
Will train until validation_0-mae hasn't improved in 10 rounds.
[1]	validation_0-mae:1.92028e+06
[2]	validation_0-mae:1.66611e+06
[3]	validation_0-mae:1.44978e+06
[4]	validation_0-mae:1.26776e+06
[5]	validation_0-mae:1.11762e+06
[6]	validation_0-mae:991517
[7]	validation_0-mae:888315
[8]	validation_0-mae:806304
[9]	validation_0-mae:739399
[10]	validation_0-mae:685741
[11]	validation_0-mae:644127
[12]	validation_0-mae:610447
[13]	validation_0-mae:585059
[14]	validation_0-mae:564825
[15]	validation_0-mae:550632
[16]	validation_0-mae:538829
[17]	validation_0-mae:529921
[18]	validation_0-mae:523079
[19]	v

[73]	validation_0-mae:484825
[74]	validation_0-mae:484817
[75]	validation_0-mae:484770
[76]	validation_0-mae:484768
[77]	validation_0-mae:484720
[78]	validation_0-mae:484672
[79]	validation_0-mae:484617
[80]	validation_0-mae:484592
[81]	validation_0-mae:484503
[82]	validation_0-mae:484434
[83]	validation_0-mae:484446
[84]	validation_0-mae:484436
[85]	validation_0-mae:484371
[86]	validation_0-mae:484301
[87]	validation_0-mae:484233
[88]	validation_0-mae:484198
[89]	validation_0-mae:484181
[90]	validation_0-mae:484110
[91]	validation_0-mae:484097
[92]	validation_0-mae:484086
[93]	validation_0-mae:484100
[94]	validation_0-mae:484010
[95]	validation_0-mae:483927
[96]	validation_0-mae:483927
[97]	validation_0-mae:483914
[98]	validation_0-mae:483896
[99]	validation_0-mae:483905
[100]	validation_0-mae:483911
[101]	validation_0-mae:483876
[102]	validation_0-mae:483872
[103]	validation_0-mae:483813
[104]	validation_0-mae:483732
[105]	validation_0-mae:483747
[106]	validation_0-mae:483725
[107]	v

[188]	validation_0-mae:490402
[189]	validation_0-mae:490420
[190]	validation_0-mae:490384
[191]	validation_0-mae:490403
[192]	validation_0-mae:490393
[193]	validation_0-mae:490389
[194]	validation_0-mae:490369
[195]	validation_0-mae:490379
[196]	validation_0-mae:490399
[197]	validation_0-mae:490403
[198]	validation_0-mae:490394
[199]	validation_0-mae:490395
[200]	validation_0-mae:490415
[201]	validation_0-mae:490398
[202]	validation_0-mae:490389
[203]	validation_0-mae:490381
[204]	validation_0-mae:490373
Stopping. Best iteration:
[194]	validation_0-mae:490369

    fold  3:  [484364.22887583]
[0]	validation_0-mae:2.20993e+06
Will train until validation_0-mae hasn't improved in 10 rounds.
[1]	validation_0-mae:1.91338e+06
[2]	validation_0-mae:1.66068e+06
[3]	validation_0-mae:1.447e+06
[4]	validation_0-mae:1.2671e+06
[5]	validation_0-mae:1.11876e+06
[6]	validation_0-mae:992425
[7]	validation_0-mae:890372
[8]	validation_0-mae:807520
[9]	validation_0-mae:740540
[10]	validation_0-mae:687900
[

[124]	valid_0's l1: 483348
[125]	valid_0's l1: 483428
[126]	valid_0's l1: 483084
[127]	valid_0's l1: 482998
[128]	valid_0's l1: 482749
[129]	valid_0's l1: 482763
[130]	valid_0's l1: 482850
[131]	valid_0's l1: 482855
[132]	valid_0's l1: 482832
[133]	valid_0's l1: 482985
[134]	valid_0's l1: 482470
[135]	valid_0's l1: 482361
[136]	valid_0's l1: 482207
[137]	valid_0's l1: 481996
[138]	valid_0's l1: 482010
[139]	valid_0's l1: 481787
[140]	valid_0's l1: 481602
[141]	valid_0's l1: 481536
[142]	valid_0's l1: 481460
[143]	valid_0's l1: 481457
[144]	valid_0's l1: 481242
[145]	valid_0's l1: 481317
[146]	valid_0's l1: 481305
[147]	valid_0's l1: 481355
[148]	valid_0's l1: 481249
[149]	valid_0's l1: 481363
[150]	valid_0's l1: 481476
[151]	valid_0's l1: 481038
[152]	valid_0's l1: 480991
[153]	valid_0's l1: 480999
[154]	valid_0's l1: 481025
[155]	valid_0's l1: 481033
[156]	valid_0's l1: 480773
[157]	valid_0's l1: 480693
[158]	valid_0's l1: 480640
[159]	valid_0's l1: 480552
[160]	valid_0's l1: 480425
[

[181]	valid_0's l1: 481774
[182]	valid_0's l1: 481912
[183]	valid_0's l1: 481843
[184]	valid_0's l1: 481750
[185]	valid_0's l1: 481815
[186]	valid_0's l1: 481897
[187]	valid_0's l1: 481815
[188]	valid_0's l1: 481836
[189]	valid_0's l1: 482021
[190]	valid_0's l1: 481915
[191]	valid_0's l1: 481939
[192]	valid_0's l1: 481962
[193]	valid_0's l1: 481797
[194]	valid_0's l1: 481851
[195]	valid_0's l1: 481791
Early stopping, best iteration is:
[180]	valid_0's l1: 481578
    fold  1:  [476572.26905489]
[1]	valid_0's l1: 1.47016e+06
Training until validation scores don't improve for 15 rounds
[2]	valid_0's l1: 1.35633e+06
[3]	valid_0's l1: 1.25734e+06
[4]	valid_0's l1: 1.16868e+06
[5]	valid_0's l1: 1.08924e+06
[6]	valid_0's l1: 1.01966e+06
[7]	valid_0's l1: 957450
[8]	valid_0's l1: 902737
[9]	valid_0's l1: 854117
[10]	valid_0's l1: 812170
[11]	valid_0's l1: 776232
[12]	valid_0's l1: 743281
[13]	valid_0's l1: 712573
[14]	valid_0's l1: 686186
[15]	valid_0's l1: 664430
[16]	valid_0's l1: 645192
[17

[155]	valid_0's l1: 475764
[156]	valid_0's l1: 475575
[157]	valid_0's l1: 475448
[158]	valid_0's l1: 475148
[159]	valid_0's l1: 475132
[160]	valid_0's l1: 475258
[161]	valid_0's l1: 475590
[162]	valid_0's l1: 475468
[163]	valid_0's l1: 475358
[164]	valid_0's l1: 475367
[165]	valid_0's l1: 475369
[166]	valid_0's l1: 475384
[167]	valid_0's l1: 475303
[168]	valid_0's l1: 475365
[169]	valid_0's l1: 475342
[170]	valid_0's l1: 475416
[171]	valid_0's l1: 475305
[172]	valid_0's l1: 475390
[173]	valid_0's l1: 475292
[174]	valid_0's l1: 475290
Early stopping, best iteration is:
[159]	valid_0's l1: 475132
    fold  3:  [476943.45170538]
[1]	valid_0's l1: 1.53367e+06
Training until validation scores don't improve for 15 rounds
[2]	valid_0's l1: 1.41573e+06
[3]	valid_0's l1: 1.31192e+06
[4]	valid_0's l1: 1.21866e+06
[5]	valid_0's l1: 1.13662e+06
[6]	valid_0's l1: 1.06528e+06
[7]	valid_0's l1: 1.00292e+06
[8]	valid_0's l1: 947656
[9]	valid_0's l1: 898837
[10]	valid_0's l1: 855843
[11]	valid_0's l1: 

StackingTransformer(estimators=[('xgboost',
                                 Pipeline(memory=None,
                                          steps=[('feature_selector',
                                                  FeatureSelector(features=['habitaciones',
                                                                            'garages',
                                                                            'banos',
                                                                            'antiguedad',
                                                                            'metroscubiertos',
                                                                            'metrostotales',
                                                                            'lat_norm',
                                                                            'lng_norm',
                                                                            'gimnasio',
                         

In [21]:
s_train = stack.transform(df_train_f.drop('precio', axis=1).values)
s_test = stack.transform(df_test_f.drop('precio', axis=1, errors='ignore').values)

Train set was detected.
Transforming...

estimator  0: [xgboost: Pipeline]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    model from fold  4: done
    ----
    DONE

estimator  1: [lightgbm: Pipeline]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    model from fold  4: done
    ----
    DONE

Transforming...

estimator  0: [xgboost: Pipeline]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    model from fold  4: done
    ----
    DONE

estimator  1: [lightgbm: Pipeline]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    model from fold  4: done
    ----
    DONE



## Prediccion con todos los features + stacking

In [22]:
df_train_s = df_train_f.copy()
df_test_s = df_test_f.copy()

df_train_s['stack01'], df_train_s['stack02'] = zip(*s_train)
df_test_s['stack01'], df_test_s['stack02'] = zip(*s_test)

features_stacking = ['stack01', 'stack02']

In [23]:
df_train_s['id'] = df_train['id']
df_test_s['id'] = df_test['id']

In [24]:
import xgboost as xgb
import pandas as pd
import numpy as np
from hyperopt import fmin, tpe, hp
from hyperopt.pyll.base import scope
import ipynb.fs.full.utils as utils
import ipynb.fs.full.features as features
import ipynb.fs.full.features_distancias as f_distancias


from sklearn.model_selection import cross_val_score
from hyperopt import STATUS_OK


features =['antiguedad', 'habitaciones', 'garages', 'banos', 'metroscubiertos',
       'metrostotales', 'lat', 'lng', 'gimnasio', 'usosmultiples',
       'piscina', 'clustering_label', 'idf_descripcion', 'porcentaje_metros', 'diferencia_metros',
       'promedio_metros_tipo_propiedad', 'prop_frecuente', 'top_provincia',
       'promedio_metros_totales_provincia', 'anio', 'mes',
       'trimestre', 'dias_desde_datos', 'cantidad_inquilinos', 'tam_ambientes', 'promedio_precio_provincia',
       'promedio_precio_ciudad', 'count_ciudad', 'promedio_id_zona',
       'count_id_zona','promedio_precio_tipo_propiedad',
       'promedio_precio_tipo_propiedad_ciudad', 'count_tipo_propiedad',
       'count_tipo_propiedad_ciudad', 'promedio_por_mes', 
       'promedio_precio_banos_garages', 'promedio_precio_hbg_tipo_propiedad',
       'promedio_precio_booleanos', 'puntaje',
       'distancia_ciudad_centrica', 'distancia_centro_mexico',
       'ranking_en_provincia_tipodepropiedad_precio',
       'ranking_en_provincia_tipodepropiedad_cantidad']


params_2nd = {
 'bagging_freq': 0,
 'max_bin': 1643,
 'max_depth': 6,
 'min_data_in_leaf': 62,
 'min_split_gain': 0.8965566444331939,
 'num_leaves': 592,
 'reg_lambda': 35.74667217035101,
#  'bagging_freq': 0
#  'max_bin': 1012,
#  'max_depth': 5,
#  'min_data_in_leaf': 40,
#  'min_split_gain': 0.2934964048344291,
#  'num_leaves': 469,
#  'reg_lambda': 36.47905035024398,
 'boosting_type': 'gbdt',
 'num_boost_round': 1200,
 'objective': 'regression',
 'metric': 'mae'
}


lgb_m_2nd = LightGBMWrapper(**params_2nd)
lgb_m_2nd.fit(utils.filtrar_features(df_train_s, features + features_stacking), df_train_s['precio'].values)

/home/camix/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's l1: 1.45636e+06
Training until validation scores don't improve for 15 rounds
[2]	valid_0's l1: 1.33137e+06
[3]	valid_0's l1: 1.22059e+06
[4]	valid_0's l1: 1.12238e+06
[5]	valid_0's l1: 1.03556e+06
[6]	valid_0's l1: 959756
[7]	valid_0's l1: 892431
[8]	valid_0's l1: 833121
[9]	valid_0's l1: 782106
[10]	valid_0's l1: 737887
[11]	valid_0's l1: 699049
[12]	valid_0's l1: 666092
[13]	valid_0's l1: 637854
[14]	valid_0's l1: 613141
[15]	valid_0's l1: 592076
[16]	valid_0's l1: 574150
[17]	valid_0's l1: 558857
[18]	valid_0's l1: 545568
[19]	valid_0's l1: 534208
[20]	valid_0's l1: 524539
[21]	valid_0's l1: 516161
[22]	valid_0's l1: 508995
[23]	valid_0's l1: 502932
[24]	valid_0's l1: 497707
[25]	valid_0's l1: 493261
[26]	valid_0's l1: 489511
[27]	valid_0's l1: 486429
[28]	valid_0's l1: 483811
[29]	valid_0's l1: 481601
[30]	valid_0's l1: 479979
[31]	valid_0's l1: 478520
[32]	valid_0's l1: 477159
[33]	valid_0's l1: 476123
[34]	valid_0's l1: 475240
[35]	valid_0's l1: 474363
[36]	valid_0'

LightGBMWrapper(bagging_freq=0, boosting_type='gbdt', class_weight=None,
                colsample_bytree=1.0, importance_type='split',
                learning_rate=0.1, max_bin=1643, max_depth=6, metric='mae',
                min_child_samples=20, min_child_weight=0.001,
                min_data_in_leaf=62, min_split_gain=0.8965566444331939,
                n_estimators=100, n_jobs=-1, num_boost_round=1200,
                num_leaves=592, objective='regression', random_state=None,
                reg_alpha=0.0, reg_lambda=35.74667217035101, silent=True,
                subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [25]:
df_test_s['target'] = lgb_m_2nd.predict(utils.filtrar_features(df_test_s, features + features_stacking))
# df_test_s = utils.pesificar_df(df_test_s, col_precio_in='target', col_precio_out='target')
df_test_s[['id', 'target']].to_csv('respuesta63.csv', index = False)

# print(f'MAE Stacking-full: {utils.MAE(df_test_s["precio"].values, df_test_s["target"].values)}')

## Prediccion solo con features de stacking

In [ ]:
params_2nd = {'bagging_fraction': 0.8243831977099841,
 'bagging_freq': int(10.0),
 'feature_fraction': 0.9228324501365147,
 'learning_rate': 0.050664243951241736,
 'max_depth': int(3.0),
 'num_leaves': int(78.0),
 'objective': 'regression',
 'boosting_type': 'dart',
 'num_boost_round': 1200,
 'metric': 'mae'}


lgb_m_2nd = LightGBMWrapper(**params_2nd)
lgb_m_2nd.fit(s_train, df_train_f['precio'].values)

df_test_f['target'] = lgb_m_2nd.predict(s_test)
df_test_f[['id', 'target']].to_csv('respuesta47.csv', index = False)

## Prediccion con promedios

In [ ]:
df_test_f['target'] = s_test.T[1].T
df_test_f[['id', 'target']].to_csv('respuesta61.csv', index = False)

In [ ]:
from scipy.stats import mode

y_pred_test = mode(s_test, axis=1)[0]
print(f"MAE Stacking only: {utils.MAE(y_pred_test, df_test_f['precio'].values)}")

In [ ]:
import numpy as np

y_pred_test = np.average(s_test, axis=1)
print(f"MAE Stacking only: {utils.MAE(y_pred_test, df_test_f['precio'].values)}")

In [ ]:
from scipy.optimize import minimize

def mae_res(weights):
    y_pred_test = np.average(s_test, weights=weights, axis=1)
    return utils.MAE(y_pred_test, df_test_f['precio'].values)

x0 = [1] * len(s_test.T)
minimize(mae_res, x0)

In [ ]:
from scipy.optimize import minimize, differential_evolution

def mae_res(weights):
    y_pred_test = np.average(s_test, weights=weights, axis=1)
    return utils.MAE(y_pred_test, df_test_f['precio'].values)

x0 = [(-3, 4)] * len(s_test.T)
differential_evolution(mae_res, bounds=x0)

## Blending

In [ ]:
from sklearn.base import clone
from sklearn.model_selection import train_test_split


class BlendingTransformer():

    def __init__(self, models, model_blend, val_split=0.15, verbose=True):
        self.models = models
        self.val_split = val_split
        self.model_blend = model_blend
        self.verbose = verbose

        
    def simple_fit(self, x, y):
        x_cat = FeatureSelector(['provincia']).transform(x)
        
        x_train, x_eval, y_train, y_eval = train_test_split(x, y, 
                                                            test_size=self.val_split,
                                                            random_state=1,
                                                            stratify=x_cat)

        self._fit_first(x_train, y_train)
        self._fit_second(x_eval, y_eval)
        


    def fit(self, x, y, x_test):

        x_cat = FeatureSelector(['provincia']).transform(x)
        
        x_train, x_eval, y_train, y_eval = train_test_split(x, y, 
                                                            test_size=self.val_split,
                                                            random_state=0)
#                                                             stratify=x_cat)

        self._fit_first(x_train, y_train)
        self._fit_second(x_eval, y_eval)

        self._fit_first(x, y)
        y_test = self._predict_second(x_test)

        x = np.concatenate((x, x_test))
        y = np.concatenate((y, y_test))
        x_cat = FeatureSelector(['provincia', 'intervalo_metros_cubiertos']).transform(x)
        
        x_train, x_eval, y_train, y_eval = train_test_split(x, y, 
                                                            test_size=self.val_split, 
                                                            random_state=0)
#                                                             stratify=x_cat)
        self._fit_first(x_train, y_train)
        self._fit_second(x_eval, y_eval)

        self._fit_first(x, y)


    def _fit_first(self, x, y):

        for i in range(len(self.models)):   
            if self.verbose:
                print(f'Fitting {i}...')
            self.models[i] = clone(self.models[i])
            self.models[i].fit(x, y)


    def _fit_second(self, x, y):
        if self.verbose:
            print(f'Fitting blender...')
        self.model_blend = clone(self.model_blend)
        self.model_blend.fit(self._predict_first(x), y)


            
    def _predict_first(self, x):
        preds = []
        for i in range(len(self.models)):
            y_pred = self.models[i].predict(x)
            preds.append(y_pred)

        return np.array(preds).T


    def _predict_second(self, x):
        return self.model_blend.predict(self._predict_first(x))


    def predict(self, x):
        return self._predict_second(x)
    
    
modelos = [
           lgb_m,
#            catboost_m,
           xgb_m,
#            forest_m
          ]

params_2nd = {'bagging_fraction': 0.8243831977099841,
 'bagging_freq': int(10.0),
 'feature_fraction': 0.9228324501365147,
 'learning_rate': 0.050664243951241736,
 'max_depth': int(3.0),
 'num_leaves': int(78.0),
 'objective': 'regression',
 'boosting_type': 'dart',
 'num_boost_round': 1200,
 'metric': 'mae'}

lgb_m_2nd = LightGBMWrapper(**params_2nd)

blend = BlendingTransformer(modelos, lgb_m_2nd)

### Simple blend

In [ ]:
blend.simple_fit(df_train_f.drop('precio', axis=1).values, df_train_f['precio'].values)

In [ ]:
utils.MAE(blend.predict(df_test_f.drop('precio', axis=1).values), df_test_f['precio'].values)

### Long blend

In [ ]:
blend.fit(df_train_f.drop('precio', axis=1).values, 
                          df_train_f['precio'].values, df_test_f.drop('precio', axis=1, errors='ignore').values)

In [ ]:
df_test_s = df_test_f.copy()
df_test_s['target'] = blend.predict(df_test_s.drop('precio', axis=1, errors='ignore').values)
# utils.MAE(df_test_s['target'], df_test_s['precio'])
# df_test_s = utils.pesificar_df(df_test_s, col_precio_in='target', col_precio_out='target')
df_test_s[['id', 'target']].to_csv('respuesta63.csv', index = False)

In [ ]:
df_test_f['precio']